In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Part 1: Load and Prepare Data

In [2]:
# Load IMDb dataset (sample)
print("Loading IMDb dataset...")
imdb_dataset = load_dataset("imdb", split="test")
imdb_sample = imdb_dataset.shuffle(seed=42).select(range(100))  # Sample 100 reviews

# Convert to DataFrame
df_imdb = pd.DataFrame({
    'text': imdb_sample['text'][:50],  # Take first 50 for annotation
    'true_label': imdb_sample['label'][:50]  # 0=negative, 1=positive
})

print(f"Loaded {len(df_imdb)} IMDb reviews")
print("\nSample review:")
print(df_imdb.iloc[0]['text'][:200] + "...")

Loading IMDb dataset...
Loaded 50 IMDb reviews

Sample review:
<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quic...


### Load manually annotated data

In [3]:
df_generated = pd.read_csv('FILE_PATH')

FileNotFoundError: [Errno 2] No such file or directory: 'FILE_PATH'

### Part 2: Load Small Qwen Model

In [ ]:
print("Loading Qwen model...")
model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # Small model suitable for annotation

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("Model loaded successfully!")
    
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have the transformers library installed and internet connection.")

Loading Qwen model...
Model loaded successfully!


### Part 3: Design Your Annotation Prompt

In [4]:
# TODO: STUDENT EXERCISE - Design the prompt template
# Instructions for students:
"""
EXERCISE 1: Design an effective prompt for sentiment analysis

Your task is to create a prompt template that will guide the Qwen model to classify 
movie reviews as either "positive" or "negative". 

Consider the following guidelines:
1. Be clear and specific about the task
2. Provide examples (few-shot learning)
3. Specify the expected output format
4. Consider edge cases (neutral reviews, mixed sentiments)

Fill in the PROMPT_TEMPLATE below:
"""

PROMPT_TEMPLATE = """
# TODO: Replace this with your designed prompt

You are a sentiment analysis expert. Classify the following movie review as either "positive" or "negative".

Examples:
Review: "This movie was amazing! Great acting and plot."
Sentiment: positive

Review: "Boring and poorly written. Waste of time."
Sentiment: negative

Now classify this review:
Review: "{text}"
Sentiment:
"""

print("Your prompt template:")
print(PROMPT_TEMPLATE.format(text="[EXAMPLE TEXT]"))

Your prompt template:

# TODO: Replace this with your designed prompt

You are a sentiment analysis expert. Classify the following movie review as either "positive" or "negative".

Examples:
Review: "This movie was amazing! Great acting and plot."
Sentiment: positive

Review: "Boring and poorly written. Waste of time."
Sentiment: negative

Now classify this review:
Review: "[EXAMPLE TEXT]"
Sentiment:



### Part 4: Annotation Function

In [6]:
def annotate_sentiment(text, max_length=512):
    """
    Annotate sentiment using the Qwen model
    
    Args:
        text (str): Review text to annotate
        max_length (int): Maximum input length
        
    Returns:
        str: Predicted sentiment ('positive' or 'negative')
    """
    # TODO: STUDENT EXERCISE - Complete this function
    """
    EXERCISE 2: Complete the annotation function
    
    Steps:
    1. Format the text using your prompt template
    2. Tokenize the input
    3. Generate model response
    4. Extract and clean the prediction
    5. Return standardized output ('positive' or 'negative')
    """
    
    try:
        # Format prompt
        prompt = PROMPT_TEMPLATE.format(text=text[:500])  # Truncate long texts
        
        # Tokenize
        inputs = tokenizer(
            prompt, 
            return_tensors="pt", 
            max_length=max_length, 
            truncation=True,
            padding=True
        ).to(device)
        
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,  # Short output expected
                temperature=0.1,    # Low temperature for consistency
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decode response
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text[len(prompt):].strip().lower()
        
        # Extract sentiment
        if "positive" in response:
            return "positive"
        elif "negative" in response:
            return "negative"
        else:
            return "unknown"  # Handle unclear responses
            
    except Exception as e:
        print(f"Error in annotation: {e}")
        return "error"

# Test the function
test_text = "This movie was absolutely wonderful! I loved every minute of it."
test_result = annotate_sentiment(test_text)
print(f"\nTest annotation:")
print(f"Text: {test_text}")
print(f"Predicted sentiment: {test_result}")

Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

Test annotation:
Text: This movie was absolutely wonderful! I loved every minute of it.
Predicted sentiment: error


### Part 5: Annotate the Datasets

In [7]:
print("\nAnnotating IMDb reviews...")
df_imdb['llm_prediction'] = df_imdb['text'].apply(annotate_sentiment)

print("\nAnnotating LLM-generated examples...")
df_generated['llm_prediction'] = df_generated['text'].apply(annotate_sentiment)

# Display results
print("\nIMDb Annotation Results:")
print(df_imdb[['text', 'true_label', 'llm_prediction']].head())

print("\nGenerated Examples Annotation Results:")
print(df_generated[['text', 'llm_prediction']].head())



Annotating IMDb reviews...
Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)
Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)
Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)
Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)
Error in annotation: Expected all tensors to be on the same device, but found at least two devices, cuda:7 and cuda:0! (when checking argument for argument index in method 

NameError: name 'df_generated' is not defined